In [1]:
cd .. 

/home/mehdi2277/Documents/HarveyMuddWork/Neural_Nets_Research/neural_nets_research


In [14]:
# EDIT THESE FIELDS TO DETERMINE HOW MANY TRAINING RUNS WE DO FOR WHAT FILE
training_rounds = 5
decoder_type = "sequence"
save_file = "t2s_1"
which_cuda = 1
print("done")

done


In [3]:
import torch.optim as optim
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import os

from neural_nets_library import training
from tree_to_sequence.tree_encoder import TreeEncoder
from tree_to_sequence.tree_decoder import TreeDecoder
from tree_to_sequence.tree_to_sequence_attention import TreeToSequenceAttention
from tree_to_sequence.grammar_tree_decoder import GrammarTreeDecoder
from tree_to_sequence.multilayer_lstm_cell import MultilayerLSTMCell
from tree_to_sequence.program_datasets import *
from tree_to_sequence.translating_trees import ( parent_to_category_FOR, category_to_child_FOR,
                                                 category_to_child_LAMBDA, 
                                                 parent_to_category_LAMBDA, ForGrammar, For,
                                                 LambdaGrammar, Lambda)
from tree_to_sequence.tree_to_tree_attention import TreeToTreeAttention
from functools import partial

In [4]:
torch.cuda.set_device(which_cuda)

In [5]:
# Counts the number of matches between the prediction and target.
def count_matches(prediction, target):
    matches = 0
    if int(prediction.value) == int(target.value):
        matches += 1
    for i in range(min(len(target.children), len(prediction.children))):
        matches += count_matches(prediction.children[i], target.children[i])
    return matches

# Program accuracy (1 if completely correct, 0 otherwise)
def program_accuracy(prediction, target):
    if decoder_type == "sequence":
        return 1 if target.tolist() == prediction else 0
    if prediction.size() == count_matches(prediction, target) and \
       prediction.size() == target.size():
        return 1
    else:
        return 0

# Calculate validation accuracy (this could either be program or token accuracy)
def validation_criterion(prediction, target):
    return program_accuracy(prediction, target)

def reset_all_parameters_uniform(model, stdev):
    for param in model.parameters():
        nn.init.uniform_(param, -stdev, stdev)

In [7]:
num_vars = 10
num_ints = 11
one_hot = False
binarize_input = True
binarize_output = False
eos_token = (decoder_type != "grammar")
long_base_case = True
input_as_seq = False
output_as_seq = (decoder_type == "sequence")
use_cuda = True

In [8]:
# Make dataset
dset_train = ForLambdaDataset("ANC/training_For.json",
                                   binarize_input=binarize_input, binarize_output=binarize_output, 
                                   eos_token=eos_token, one_hot=one_hot,
                                   num_ints=num_ints, num_vars=num_vars,
                                   long_base_case=long_base_case, 
                                   input_as_seq=input_as_seq, 
                                   output_as_seq=output_as_seq)

dset_val = ForLambdaDataset("ANC/validation_For.json",
                                   binarize_input=binarize_input, binarize_output=binarize_output, 
                                   eos_token=eos_token, one_hot=one_hot,
                                   num_ints=num_ints, num_vars=num_vars,
                                   long_base_case=long_base_case, 
                                   input_as_seq=input_as_seq, 
                                   output_as_seq=output_as_seq)

dset_test = ForLambdaDataset("ANC/test_For.json",
                                   binarize_input=binarize_input, binarize_output=binarize_output, 
                                   eos_token=eos_token, one_hot=one_hot,
                                   num_ints=num_ints, num_vars=num_vars,
                                   long_base_case=long_base_case, 
                                   input_as_seq=input_as_seq, 
                                   output_as_seq=output_as_seq)

In [9]:
max_size = int(max([len(x[1]) for x in dset_train] + [len(x[1]) for x in dset_val] + [len(x[1]) for x in dset_test]))

embedding_size = 256
hidden_size = 256
num_layers = 1
alignment_size = 50
align_type = 1
encoder_input_size = num_vars + num_ints + len(for_ops)
nclass = num_vars + num_ints + len(lambda_ops)
plot_every = 100
num_categories = len(LambdaGrammar)
num_possible_parents = len(Lambda)
save_every=5000
max_num_children = 2 if binarize else 4

In [10]:
def save_plots():
    # Save plots
    plt.plot([x * plot_every for x in range(len(train_plot_losses))], train_plot_losses)
    plt.plot([x * plot_every for x in range(len(val_plot_losses))], val_plot_losses)
    plt.title("Training Loss")
    plt.xlabel('Training Examples')
    plt.ylabel('Cross-Entropy Loss')
    plt.legend(("Train", "Validation"))
    plt.savefig(save_folder + "/" + save_file + "_train_plot.png")
    plt.close()

    plt.plot([x * plot_every for x in range(len(train_plot_accuracies))], train_plot_accuracies)
    plt.plot([x * plot_every for x in range(len(val_plot_accuracies))], val_plot_accuracies)
    plt.title("Training Program Accuracy")
    plt.xlabel('Training Examples')
    plt.ylabel('Percent Accurate Programs')
    plt.legend(("Train", "Validation"))
    plt.savefig(save_folder + "/" + save_file + "_accuracy_plot.png")

In [11]:
def save_test_accuracies():
    with open(save_folder + "/" + save_file + "_test.txt", "a") as file:
        for val in test_accuracies:
            file.write(str(val) + ",")

In [12]:
def make_model():
    encoder = TreeEncoder(encoder_input_size, hidden_size, num_layers, [1, 2, 3, 4, 5], attention=True, one_hot=one_hot)

    if decoder_type == "grammar":
        decoder = GrammarTreeDecoder(embedding_size, hidden_size, num_categories, 
                             num_possible_parents, 
                             partial(parent_to_category_LAMBDA, num_vars, num_ints), 
                             partial(category_to_child_LAMBDA, num_vars, num_ints), 
                             share_linear=True, share_lstm_cell=True, 
                             num_ints_vars=num_ints + num_vars)
        
        program_model = TreeToTreeAttention(encoder, decoder, 
                                        hidden_size, embedding_size, 
                                        nclass=nclass, root_value=nclass,
                                        alignment_size=alignment_size, 
                                        align_type=align_type, max_size=max_size)
        
    elif decoder_type == "tree":
        decoder = TreeDecoder(embedding_size, hidden_size, max_num_children, nclass=nclass)
        
        program_model = TreeToTreeAttention(encoder, decoder, 
                                        hidden_size, embedding_size, 
                                        nclass=nclass, root_value=nclass,
                                        alignment_size=alignment_size, 
                                        align_type=align_type, max_size=max_size)
        
    elif decoder_type == "sequence":
        decoder = MultilayerLSTMCell(embedding_size + hidden_size, hidden_size, num_layers)
        
        program_model = TreeToSequenceAttention(encoder, decoder, hidden_size, nclass, 
                                                embedding_size, alignment_size=alignment_size, 
                                                align_type=align_type)
        
    if use_cuda:
        program_model = program_model.cuda()
        
    reset_all_parameters_uniform(program_model, 0.1)
    encoder.initialize_forget_bias(3)
    decoder.initialize_forget_bias(3)
    
    return program_model

In [ ]:
test_accuracies = []
program_model = make_model()
save_folder = "test_various_models"

# Optimizer
optimizer = optim.Adam(program_model.parameters(), lr=0.005)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=500, factor=0.8)

# Train
os.system("mkdir "+ save_folder)
best_model, train_plot_losses, train_plot_accuracies, val_plot_losses, val_plot_accuracies = training.train_model_tree_to_tree(
                                program_model, 
                                dset_train, 
                                optimizer, 
                                lr_scheduler=lr_scheduler, 
                                num_epochs=5, plot_every=plot_every,
                                batch_size=100, 
                                print_every=200, 
                                validation_dset = dset_val,                                                          
                                validation_criterion=validation_criterion,
                                use_cuda=use_cuda, 
                                deep_copy_desired=False,
                                plateau_lr=True,
                                save_file=save_file,
                                save_folder=save_folder,
                                save_every=save_every)

# Test
test_accuracy = training.test_model_tree_to_tree(best_model, dset_test, validation_criterion) #TODO: change where best_model is saved
test_accuracies.append(test_accuracy)
save_plots()
    
save_test_accuracies()

Epoch 0/4
----------
